<a href="https://colab.research.google.com/github/TyrGunllod/TELECOMX_ML/blob/main/telecomx_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de bibliotecas

# Extração do arquivo tratado

# Encoding

# Verificação da Proporção de Evasão

# Balanceamento de Classes (opcional )